Question -3
How does the price changes for neighbourhood throughout the year?
1. Plotting line graph to see the variation (Use average price of the same day to calculate the data)


In [1]:
# Imports
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import folium
import json
import re
from collections import Counter
from scipy import stats
import geopandas as gpd

In [2]:
listings_filepath = '/Users/abhayrakeshdeshmukhmaurya/Library/CloudStorage/OneDrive-UniversityofEdinburgh/CS/University/Year 2/Data Science/CW3/AirBnb/datasets/listings.csv'
listings_df = pd.read_csv(listings_filepath)

In [3]:
calendar_filepath = '/Users/abhayrakeshdeshmukhmaurya/Library/CloudStorage/OneDrive-UniversityofEdinburgh/CS/University/Year 2/Data Science/CW3/AirBnb/datasets/calendar.csv'
calendar_df = pd.read_csv(calendar_filepath)

In [4]:
neighbourhoods_filepath = '/Users/abhayrakeshdeshmukhmaurya/Library/CloudStorage/OneDrive-UniversityofEdinburgh/CS/University/Year 2/Data Science/CW3/AirBnb/datasets/neighbourhoods.csv'
neighbourhoods_df = pd.read_csv(neighbourhoods_filepath)

In [5]:
listings_for_calendar = listings_df.loc[:, ['id', 'neighbourhood_cleansed', 'latitude', 'longitude']]

In [6]:
calendar_df['listing_id'].nunique()

7389

In [7]:
# calendar_df = calendar_df['price'].dropna()
# calendar_df['price'] = calendar_df['price'].str.replace('$', '')  # Remove dollar sign
# calendar_df['price'] = calendar_df['price'].str.replace(',', '')  # Remove comma separators (if present)
# calendar_df['price'] = calendar_df['price'].astype(float).astype(int)  # Convert to float and then to int

calendar_df['price'] = calendar_df['price'].str.replace(',', '',regex=True).str.replace('$', '',regex=True).astype('float')


# calendar_df['date'] = pd.to_datetime(calendar_df['date'])
# # Group the data by date and calculate the mean price for each date
# avg_price_by_date = calendar_df.groupby('date')['price'].mean()

In [8]:
# Q1 = calendar_df['price'].quantile(0.25)
# Q3 = calendar_df['price'].quantile(0.75)
# IQR = Q3 - Q1

# # Define the lower and upper bounds for outlier detection
# lower_bound = Q1 - 1.5*IQR
# upper_bound = Q3 + 1.5*IQR

# # Identify and remove outliers
# calendar_df = calendar_df[(calendar_df['price'] >= lower_bound) & (calendar_df['price'] <= upper_bound)]

In [9]:
pivot_df = calendar_df.pivot(index='listing_id', columns='date', values='price')
pivot_df.to_csv('../datasets/pivot_df.csv')
pivot_df['2022-12-18']


OSError: Cannot save file into a non-existent directory: '../datasets'

In [ ]:
# merge the two dataframes on the id and listing_id columns
merged_df = pd.merge(pivot_df, listings_for_calendar, left_on='listing_id', right_on='id')
merged_df
# check if the id and listing_id columns match
# match = all(merged_df['id'] == merged_df['listing_id'])

# print(match)

,2022-12-16,2022-12-17,2022-12-18,2022-12-19,2022-12-20,2022-12-21,2022-12-22,2022-12-23,2022-12-24,2022-12-25,...,2023-12-11,2023-12-12,2023-12-13,2023-12-14,2023-12-15,2023-12-16,id,neighbourhood_cleansed,latitude,longitude
0,100.0,115.0,115.0,115.0,115.0,120.0,120.0,120.0,130.0,130.0,...,120.0,120.0,120.0,120.0,120.0,NaN,15420,"Old Town, Princes Street and Leith Street",55.957590,-3.188050
1,NaN,79.0,73.0,65.0,65.0,65.0,65.0,67.0,65.0,65.0,...,93.0,93.0,93.0,94.0,94.0,94.0,24288,"Canongate, Southside and Dumbiedykes",55.943830,-3.184450
2,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,...,55.0,55.0,55.0,55.0,55.0,NaN,38628,Joppa,55.942150,-3.096400
3,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,...,45.0,45.0,45.0,45.0,45.0,NaN,44009,Marchmont West,55.938000,-3.194680
4,NaN,42.0,44.0,44.0,44.0,44.0,44.0,42.0,42.0,56.0,...,39.0,39.0,39.0,39.0,42.0,42.0,44552,South Leith,55.966120,-3.171840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7384,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,295.0,350.0,...,350.0,350.0,350.0,350.0,350.0,NaN,782136800346144859,New Town East and Gayfield,55.958031,-3.184608
7385,NaN,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,782448596836883109,Leith (Albert Street),55.963253,-3.172172
7386,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,...,35.0,35.0,35.0,35.0,35.0,NaN,782474793883477230,The Inch,55.917450,-3.162740
7387,75.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,100.0,100.0,...,75.0,75.0,75.0,75.0,75.0,NaN,782510024659134909,Bonnington,55.974178,-3.181042


In [ ]:

#Load the dataset
# quality_df = pd.read_csv('quality_data.csv')

# #Calculate the IQR
# Q1 = quality_df['price'].quantile(0.25)
# Q3 = quality_df['price'].quantile(0.75)
# IQR = Q3 - Q1

# # Define the lower and upper bounds for outlier detection
# lower_bound = Q1 - 1.5*IQR
# upper_bound = Q3 + 1.5*IQR

# # Identify and remove outliers
# quality_df = quality_df[(quality_df['price'] >= lower_bound) & (quality_df['price'] <= upper_bound)]

In [ ]:
# pivot_df = pivot_df[~(pivot_df > 300).any(axis=1)]
# pivot_df

# merged_df = merged_df[~(merged_df > 300).any(axis=1)]
# merged_df

# int_cols = merged_df.select_dtypes(exclude=['int', 'float']).columns.tolist()
# print(int_cols)

# # drop the rows where any column has a value greater than 200, except for the non-int columns
# merged_df = merged_df[merged_df[int_cols].apply(lambda x: ~(x > 200).any(), axis=1)]

# specify the columns to exclude
cols_to_exclude = ['id', 'neighbourhood_cleansed', 'latitude', 'longitude']

# select all columns except the excluded ones
non_excluded_cols = [col for col in merged_df.columns if col not in cols_to_exclude]

# drop the rows where any column except the excluded ones has a value greater than 200
merged_df = merged_df[merged_df[non_excluded_cols].apply(lambda x: ~(x > 200).any(), axis=1)]
merged_df

,2022-12-16,2022-12-17,2022-12-18,2022-12-19,2022-12-20,2022-12-21,2022-12-22,2022-12-23,2022-12-24,2022-12-25,...,2023-12-11,2023-12-12,2023-12-13,2023-12-14,2023-12-15,2023-12-16,id,neighbourhood_cleansed,latitude,longitude
0,100.0,115.0,115.0,115.0,115.0,120.0,120.0,120.0,130.0,130.0,...,120.0,120.0,120.0,120.0,120.0,NaN,15420,"Old Town, Princes Street and Leith Street",55.957590,-3.188050
1,NaN,79.0,73.0,65.0,65.0,65.0,65.0,67.0,65.0,65.0,...,93.0,93.0,93.0,94.0,94.0,94.0,24288,"Canongate, Southside and Dumbiedykes",55.943830,-3.184450
2,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,...,55.0,55.0,55.0,55.0,55.0,NaN,38628,Joppa,55.942150,-3.096400
3,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,...,45.0,45.0,45.0,45.0,45.0,NaN,44009,Marchmont West,55.938000,-3.194680
4,NaN,42.0,44.0,44.0,44.0,44.0,44.0,42.0,42.0,56.0,...,39.0,39.0,39.0,39.0,42.0,42.0,44552,South Leith,55.966120,-3.171840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7383,NaN,99.0,93.0,93.0,93.0,93.0,93.0,99.0,99.0,93.0,...,93.0,93.0,93.0,93.0,99.0,99.0,781855194633396192,Pilrig,55.966882,-3.175113
7385,NaN,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,782448596836883109,Leith (Albert Street),55.963253,-3.172172
7386,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,...,35.0,35.0,35.0,35.0,35.0,NaN,782474793883477230,The Inch,55.917450,-3.162740
7387,75.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,100.0,100.0,...,75.0,75.0,75.0,75.0,75.0,NaN,782510024659134909,Bonnington,55.974178,-3.181042


In [ ]:
# merged_df = merged_df[(merged_df > 300).any(axis=1)]
# merged_df

In [ ]:
for cols in merged_df.columns:
    print(merged_df[cols].isna().count())

4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059
4059


In [ ]:
# pivot_df.dropna(inplace=True)
merged_df = merged_df.fillna(merged_df.median()) 
merged_df


/var/folders/g6/l9c_w0xj7zv_8_35j60mvnm80000gp/T/ipykernel_9324/1688827338.py:2: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  merged_df = merged_df.fillna(merged_df.median())


,2022-12-16,2022-12-17,2022-12-18,2022-12-19,2022-12-20,2022-12-21,2022-12-22,2022-12-23,2022-12-24,2022-12-25,...,2023-12-11,2023-12-12,2023-12-13,2023-12-14,2023-12-15,2023-12-16,id,neighbourhood_cleansed,latitude,longitude
0,100.0,115.0,115.0,115.0,115.0,120.0,120.0,120.0,130.0,130.0,...,120.0,120.0,120.0,120.0,120.0,95.0,15420,"Old Town, Princes Street and Leith Street",55.957590,-3.188050
1,80.0,79.0,73.0,65.0,65.0,65.0,65.0,67.0,65.0,65.0,...,93.0,93.0,93.0,94.0,94.0,94.0,24288,"Canongate, Southside and Dumbiedykes",55.943830,-3.184450
2,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,...,55.0,55.0,55.0,55.0,55.0,95.0,38628,Joppa,55.942150,-3.096400
3,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,...,45.0,45.0,45.0,45.0,45.0,95.0,44009,Marchmont West,55.938000,-3.194680
4,80.0,42.0,44.0,44.0,44.0,44.0,44.0,42.0,42.0,56.0,...,39.0,39.0,39.0,39.0,42.0,42.0,44552,South Leith,55.966120,-3.171840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7383,80.0,99.0,93.0,93.0,93.0,93.0,93.0,99.0,99.0,93.0,...,93.0,93.0,93.0,93.0,99.0,99.0,781855194633396192,Pilrig,55.966882,-3.175113
7385,80.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,782448596836883109,Leith (Albert Street),55.963253,-3.172172
7386,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,...,35.0,35.0,35.0,35.0,35.0,95.0,782474793883477230,The Inch,55.917450,-3.162740
7387,75.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,100.0,100.0,...,75.0,75.0,75.0,75.0,75.0,95.0,782510024659134909,Bonnington,55.974178,-3.181042


In [ ]:
merged_df.dropna(inplace=True)
merged_df

,2022-12-16,2022-12-17,2022-12-18,2022-12-19,2022-12-20,2022-12-21,2022-12-22,2022-12-23,2022-12-24,2022-12-25,...,2023-12-11,2023-12-12,2023-12-13,2023-12-14,2023-12-15,2023-12-16,id,neighbourhood_cleansed,latitude,longitude
0,100.0,115.0,115.0,115.0,115.0,120.0,120.0,120.0,130.0,130.0,...,120.0,120.0,120.0,120.0,120.0,95.0,15420,"Old Town, Princes Street and Leith Street",55.957590,-3.188050
1,80.0,79.0,73.0,65.0,65.0,65.0,65.0,67.0,65.0,65.0,...,93.0,93.0,93.0,94.0,94.0,94.0,24288,"Canongate, Southside and Dumbiedykes",55.943830,-3.184450
2,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,...,55.0,55.0,55.0,55.0,55.0,95.0,38628,Joppa,55.942150,-3.096400
3,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,...,45.0,45.0,45.0,45.0,45.0,95.0,44009,Marchmont West,55.938000,-3.194680
4,80.0,42.0,44.0,44.0,44.0,44.0,44.0,42.0,42.0,56.0,...,39.0,39.0,39.0,39.0,42.0,42.0,44552,South Leith,55.966120,-3.171840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7383,80.0,99.0,93.0,93.0,93.0,93.0,93.0,99.0,99.0,93.0,...,93.0,93.0,93.0,93.0,99.0,99.0,781855194633396192,Pilrig,55.966882,-3.175113
7385,80.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,782448596836883109,Leith (Albert Street),55.963253,-3.172172
7386,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,...,35.0,35.0,35.0,35.0,35.0,95.0,782474793883477230,The Inch,55.917450,-3.162740
7387,75.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,100.0,100.0,...,75.0,75.0,75.0,75.0,75.0,95.0,782510024659134909,Bonnington,55.974178,-3.181042


In [ ]:
# Q1 = calendar_df['price'].quantile(0.25)
# Q3 = calendar_df['price'].quantile(0.75)
# IQR = Q3 - Q1

# # Define the lower and upper bounds for outlier detection
# lower_bound = Q1 - 1.5*IQR
# upper_bound = Q3 + 1.5*IQR

# filtered_df = pivot_df[~((pivot_df < (Q1 - 1.5 * IQR)) | (pivot_df > (Q3 + 1.5 * IQR))).any(axis=1)]
# filtered_df

In [ ]:
grouped_df = merged_df.groupby('neighbourhood_cleansed').mean()
grouped_df

,2022-12-16,2022-12-17,2022-12-18,2022-12-19,2022-12-20,2022-12-21,2022-12-22,2022-12-23,2022-12-24,2022-12-25,...,2023-12-10,2023-12-11,2023-12-12,2023-12-13,2023-12-14,2023-12-15,2023-12-16,id,latitude,longitude
neighbourhood_cleansed,,,,,,,,,,,,,,,,,,,,,
Abbeyhill,83.000000,84.728814,80.457627,81.033898,80.932203,80.491525,80.745763,84.152542,86.406780,86.355932,...,87.525424,87.186441,87.169492,87.169492,87.237288,89.542373,94.745763,2.595244e+17,55.955405,-3.167248
Baberton and Juniper Green,80.142857,80.142857,77.142857,77.142857,77.142857,77.142857,77.142857,80.142857,80.142857,84.428571,...,81.571429,81.571429,81.571429,81.571429,81.571429,81.571429,95.000000,3.635418e+17,55.907795,-3.285446
Balerno and Bonnington Village,79.571429,79.571429,78.857143,78.857143,78.857143,78.857143,73.142857,73.857143,73.857143,73.142857,...,84.142857,84.142857,84.142857,84.142857,84.142857,84.857143,95.000000,2.653759e+07,55.876611,-3.353850
Balgreen and Roseburn,111.366667,117.966667,115.033333,114.366667,114.366667,114.366667,115.233333,118.866667,118.733333,118.233333,...,123.166667,122.966667,122.966667,122.966667,122.966667,123.633333,98.333333,2.393760e+17,55.942864,-3.248113
"Barnton, Cammo and Cramond South",127.500000,138.750000,138.750000,138.750000,138.750000,138.750000,138.750000,138.750000,140.000000,140.000000,...,151.000000,151.250000,151.250000,151.250000,151.250000,151.000000,95.000000,3.250945e+17,55.963433,-3.306517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Trinity,80.242424,80.393939,77.606061,77.303030,77.303030,76.909091,76.909091,78.939394,79.272727,76.424242,...,78.515152,78.424242,78.424242,78.424242,78.424242,81.303030,95.000000,1.913119e+17,55.975047,-3.210082
Trinity East and The Dudleys,108.173913,108.869565,102.304348,100.521739,99.782609,100.391304,101.739130,108.086957,108.086957,107.086957,...,104.739130,104.608696,104.608696,104.608696,104.565217,112.347826,95.000000,2.372368e+17,55.975124,-3.194771
West Pilton,55.958333,56.583333,52.166667,51.333333,51.750000,51.458333,51.791667,56.416667,57.666667,55.041667,...,59.500000,59.041667,59.166667,59.166667,59.250000,62.416667,95.000000,2.134741e+17,55.971147,-3.241764
